In [1]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [2]:
import warnings
warnings.filterwarnings('ignore')
from transformers import (
    LlamaForCausalLM,
    pipeline,
    AutoTokenizer,AutoModelForCausalLM,
    LlamaModel,
    LlamaConfig, Qwen2VLForConditionalGeneration
)
import transformers
import json

from datasets import load_dataset
import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

In [3]:
model_dir = "/data/Models/llama3_8B_Base"
dataset_dir = "/data/Datasets/nyu-mll_glue"

In [4]:
# 加载 GLUE 数据集，默认为 'sst2' 任务，可以换成其他任务如 'mnli'、'qqp' 等
dataset = load_dataset(dataset_dir, "cola")
# 获取测试集
test_set = dataset["test"]
# 打印测试集的前10条数据
# test_set
# for i in range(10):
#     print(test_set[i])
test_sentences = [test_set[i]['sentence'] for i in range(30)]
test_sentences

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, truncation=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

base_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    load_in_8bit=False,
    torch_dtype=torch.float32,
    device_map='auto',
)

In [6]:
# base_model = LlamaForCausalLM.from_pretrained(
#     model_dir,
#     load_in_8bit=False,
#     torch_dtype=torch.float16,
#     device_map='auto',
# )

In [7]:
intermediate_outputs = {}

In [8]:
def hook_func(module, inputs, outputs, layer_idx):
    intermediate_outputs[layer_idx] = outputs[0]

In [9]:
for layer_idx, layer in enumerate(base_model.model.layers):
    layer.register_forward_hook(
        lambda m, inp, out, idx=layer_idx: hook_func(m, inp, out, idx)
    )

In [10]:
question = "Introduce UT Austin to me"

inputs = tokenizer(test_sentences, padding=True, truncation=True, return_tensors='pt').to(base_model.device)

with torch.no_grad():
    response = base_model.generate(inputs.input_ids, max_length=128)
# print(response)
# print(tokenizer.decode(response.cpu()[0], skip_special_tokens=True))

In [11]:
# for layer_idx, tensor in intermediate_outputs.items():
#     print(f"Layer {layer_idx}: {tensor.shape}")

In [12]:
intermediate_outputs[0][:,0,:].shape

## Zero-dimension Persistent Homology

In [13]:
import numpy as np
import scipy.spatial.distance as dist
import gudhi as gd
import matplotlib.pyplot as plt
from persim import plot_diagrams
from ripser import ripser
from ripser import Rips
import persim

In [14]:
def compute_rf_vr(pm_data):
    rips = Rips(maxdim=2)
    dgm = rips.fit_transform(pm_data)
    H0 = dgm[0][:-1]
    if len(H0) > 1:
        r_f = np.max(H0[:, 1])  # 取最大的死亡时间（即最小连通分量合并半径）
    else:
        r_f = 0  # 如果只有一个分量，说明已经连通
    return r_f, dgm

In [15]:
pm_data = np.array(intermediate_outputs[26][:,0,:].cpu())

# Instantiate Vietoris-Rips solver
rips = Rips(maxdim=2)
dgm = rips.fit_transform( pm_data)

plt.figure(figsize=(5, 5), dpi=80)
plt.rcParams.update({'font.size': 15})
persim.plot_diagrams(dgm, title="Persistence Diagram")
# print(f"rips.r_cover_ is ... {rips.r_cover_}")
# plt.savefig("homology_example_persistence-diagram.png", dpi='figure', format=None, metadata=None,
#         bbox_inches=None, pad_inches=0.1,
#         facecolor='white', edgecolor='auto')

In [16]:
r_f_list = []
for layer_idx, tensor in intermediate_outputs.items():
    # 计算最小半径
    r_f, _ = compute_rf_vr(tensor[:, 0, :].cpu())
    r_f_list.append(r_f)
    # print(f"Layer {layer_idx} -- radis -- {r_f}")

In [23]:
#---------------------------------------------------------------------------------------------------
p_num = len(base_model.model.layers)
x = np.linspace(0, p_num, 1)  # 在 0 到 10 之间生成 100 个等间距的点
x = np.arange(p_num)
y = r_f_list  

plt.plot(x, y, label="Llama3-8B-Base", color='b', linestyle='-')
plt.scatter(x, y, color='r', marker='o')
plt.title("Zero-dimension Persistent Homology")
plt.xlabel("Layers")
plt.ylabel("Radius")

plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# LLM_model = LlamaForCausalLM.from_pretrained(
#     model_dir,
#     load_in_8bit=False,
#     torch_dtype=torch.float16,
#     device_map='auto',
# )